# Gravitationally Bound System

For the two-body gravitationally-interacting system, the Lagrangian is given by the equation:

$$\begin{align}
  \mathcal{L} = \frac12 m_1 (\dot{x}_1^2 + \dot{y}_1^2)
              + \frac12 m_2 (\dot{x}_2^2 + \dot{y}_2^2)
              + \frac{G m_1 m_2}{r},
\end{align}$$

where
$\begin{align}
    r = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}.
\end{align}$

This equation has eight variables that we care about—each position component and its associated first derivative—that will be the generalized coordinates of the Euler-Lagrange equations. For example, we have

$\begin{align}
    \frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{x}_1}
        =
    \frac{\partial \mathcal{L}}{\partial x_1}
\end{align}$

and so on. These will be put into an 8-component vector, $\vec{y}$, with each of the components followed by its first derivative, and that vector along with $d\vec{y}/dt$ will be used with solve_ivp to find the long-term motion of the system.

In [ ]:
%matplotlib notebook

import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
class LagrangianGravitySystem():
    """
    implements the parameters and Lagrange's equations for a
    gravitationally-bound system
     
    Parameters
    ----------
    m1   | float       | mass of first body
    m2   | float       | mass of second body
    pos1 | float array | initial (x,y) position of body 1
    pos2 | float array | initial (x,y) position of body 2
    v1   | float array | initial (x,y) velocity of body 1
    v2   | float array | initial (x,y) velocity of body 2

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, m1, m2, pos1, pos2, v1, v2, G=2.
                ):
        self.G  = G # gravity constant cgs
        self.m1 = m1
        self.m2 = m2
        self.p1 = pos1
        self.p2 = pos2
        self.v1 = v1
        self.v2 = v2
        self.r  = np.sqrt( (self.p1[0]-self.p2[0])**2 + (self.p1[1]-self.p2[1])**2 )
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [x1, vx1, y1, vy1, x2, vx2, y2, vy2]
        
        Parameters
        ----------
        t | float       | time
        y | float array | An 8 component vector for each position and velocity component
            
        """
        r = np.sqrt ((y[0] - y[4])**2 + (y[2] - y[6])**2 )
        return [y[1], -(self.G * self.m2 / r**(3.)) * (y[0] - y[4]),\
                y[3], -(self.G * self.m2 / r**(3.)) * (y[2] - y[6]),\
                y[5],  (self.G * self.m1 / r**(3.)) * (y[0] - y[4]),\
                y[7],  (self.G * self.m1 / r**(3.)) * (y[2] - y[6])]
    
    def solve_ode(self, t_pts, 
                  abserr=1.0e-12, relerr=1.0e-12):
        """
        Solve the ODE.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [self.p1[0], self.v1[0], self.p1[1], self.v1[1],\
             self.p2[0], self.v2[0], self.p2[1], self.v2[1]] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        y_new = solution.y

        return y_new

We are going to create an object from this class from the center of mass frame for an object orbiting another with 100 times its mass and then animate this system.

In [ ]:
lgs = LagrangianGravitySystem (m1=100., m2=1., pos1=[0,0], pos2=[1.5,0], v1=[0,-10./100.], v2=[0,10], G=1.)
tpts = np.arange (0,30.01,0.01)
vec = lgs.solve_ode (t_pts=tpts)

In [ ]:
%%capture

fig_anim = plt.figure()
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_xlim(-5., 2.)
ax_anim.set_ylim(-3., 3.)
ax_anim.set_aspect (1)
ax_anim.axis ('off')

# By assigning the first return from plot to line_anim, we can later change
#  the values in the line.
line_anim1, = ax_anim.plot(vec[4], 
                            vec[6],
                            color='#bb0000', lw=0.5)
pt1_anim, = ax_anim.plot (vec[4], vec[6], marker='o', color='#bb0000', markersize=8.)

line_anim2, = ax_anim.plot(vec[0], 
                            vec[2],
                            color='#666666', lw=0.5)
pt2_anim, = ax_anim.plot(vec[0], vec[2], marker='o', color='#666666', markersize=8.)

fig_anim.tight_layout()

In [ ]:
def animate_gravity(i):
    """This is the function called by FuncAnimation to create each frame,
        numbered by i.  So each i corresponds to a point in the t_pts
        array, with index i.
    """
    i = 4*i
    line_anim1.set_data (vec[4], vec[6])  # overwrite line_anim with new points
    pt1_anim.set_data (vec[4][i], vec[6][i])
    line_anim2.set_data (vec[0], vec[2])  # overwrite line_anim with new points
    pt2_anim.set_data (vec[0][i], vec[2][i])
    return (line_anim1,)   # this is needed for blit=True to work

In [ ]:
frame_interval = 30  # time between frames
frame_number = int (len (vec[6])/4)   # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim, 
                               animate_gravity, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())

We can see that, in this center of mass frame with one massive object, the outer object traces an ellipse, with the other approximately at one of that ellipses' foci, just as we saw with the orbits in the unit on energy.

### Some other fun orbit:

In [ ]:
lgs = LagrangianGravitySystem (m1=2., m2=1., pos1=[-0.5,0], pos2=[0.5,0], v1=[0,-1.], v2=[0,2.], G=2.)
tpts = np.arange (0,30.01,0.01)
vec = lgs.solve_ode (t_pts=tpts)

In [ ]:
%%capture

fig_anim = plt.figure()
ax_anim = fig_anim.add_subplot(1,1,1)
#ax_anim.set_xlim(-5., 2.)
#ax_anim.set_ylim(-3., 3.)
ax_anim.set_aspect (1)
ax_anim.axis ('off')

# By assigning the first return from plot to line_anim, we can later change
#  the values in the line.
line_anim1, = ax_anim.plot(vec[4], 
                            vec[6],
                            color='#666666', lw=0.5)
pt1_anim, = ax_anim.plot (vec[4], vec[6], marker='o', color='#666666', markersize=8.)

line_anim2, = ax_anim.plot(vec[0], 
                            vec[2],
                            color='#bb0000', lw=0.5)
pt2_anim, = ax_anim.plot(vec[0], vec[2], marker='o', color='#bb0000', markersize=8.)

fig_anim.tight_layout()

In [ ]:
def animate_gravity(i):
    """This is the function called by FuncAnimation to create each frame,
        numbered by i.  So each i corresponds to a point in the t_pts
        array, with index i.
    """
    i = 4*i
    line_anim1.set_data (vec[4], vec[6])  # overwrite line_anim with new points
    pt1_anim.set_data (vec[4][i], vec[6][i])
    line_anim2.set_data (vec[0], vec[2])  # overwrite line_anim with new points
    pt2_anim.set_data (vec[0][i], vec[2][i])
    return (line_anim1,)   # this is needed for blit=True to work

In [ ]:
frame_interval = 30  # time between frames
frame_number = int (len (vec[6])/4)   # number of frames to include (index of t_pts)
anim2 = animation.FuncAnimation(fig_anim, 
                               animate_gravity, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim2.to_jshtml())

Again, here, each object orbits the center of mass in an ellipse.

We also can see from this second animation that both objects reach the extremes of their orbits at the same time.